<a href="https://colab.research.google.com/github/GuptaNavdeep1983/CS688/blob/main/Assignment_6.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:

https://datascience.stackexchange.com/questions/54904/how-to-avoid-tokenizing-w-sklearn-feature-extraction


https://www.datacamp.com/community/tutorials/discovering-hidden-topics-python

https://www.freecodecamp.org/news/an-introduction-to-bag-of-words-and-how-to-code-it-in-python-for-nlp-282e87a9da04/

https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html#sklearn.feature_extraction.text.CountVectorizer.transform

In [ ]:
pip install gensim

In [ ]:
import statsmodels.api as sm 
import pylab as py 
import matplotlib.pyplot as plt
import pandas as pd
import urllib.request
import datetime
from zipfile import ZipFile
from gzip import decompress
from json import loads
from requests import get
import requests, zipfile, io
from bs4 import BeautifulSoup
import math
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import re

from gensim import corpora
from gensim.models import LsiModel
from gensim.models.coherencemodel import CoherenceModel

In [ ]:
def get_crowd_funding_data_links(filter_by_years):
    # get the content of the website
    content = urllib.request.urlopen('https://webrobots.io/indiegogo-dataset/').read()
    soup = BeautifulSoup(content,features='html.parser')
    
    all_stories = pd.DataFrame([], columns=['Year', 'Month', 'json', 'csv'])
    parent_div = soup.find(name='div', attrs={'class':'fusion-text'})

    # Iterate through the divs to find the list of Month-Year wise anchor tags
    index = 0
    for year_data in parent_div.find_all(name='ul'):
        all_months = year_data.find_all('li')
        for month in all_months:
            complete_text = month.text
            date_time_obj = datetime.datetime.strptime((complete_text.split('[')[0]).strip(), '%Y-%m-%d')
            if date_time_obj.year in filter_by_years:
                all_stories.loc[index, "Year"] = date_time_obj.year
                all_stories.loc[index, "Month"] = str(date_time_obj.month) if date_time_obj.month >=10 else f'0{date_time_obj.month}'
                all_stories.loc[index, "json"] = month.find_all('a')[0]["href"]
                all_stories.loc[index, "csv"] = month.find_all('a')[1]["href"]
                index = index + 1
    return all_stories

def get_current_index_data(df, indx, columns):
    r = requests.get(df["csv"][indx])
    z = zipfile.ZipFile(io.BytesIO(r.content))
    csv_data = z.open("Indiegogo.csv")
    data_df = pd.read_csv(csv_data, usecols=columns, dtype={'title':str, 'tagline':str})
    return data_df

In [ ]:
df = get_crowd_funding_data_links([2020])

documents = []
for indx in df.head(5).index:
    current_month_data = get_current_index_data(df, indx, ["title", "tagline"])
    current_month_data.dropna(axis=0, inplace=True)
    only_taglines = current_month_data["tagline"]
    all_content = ''.join(str(e) for e in only_taglines)
    documents.append(all_content)

In [153]:
df = pd.DataFrame(documents, columns=['content'])
vectorizer = CountVectorizer(input='content',stop_words='english', analyzer='word', token_pattern=r'\b[a-zA-Z]{2,}\b',max_features=100)
vectorizer.fit_transform(df['content'])
doc_term_matrix = vectorizer.transform(df['content'])
feature_names = vectorizer.get_feature_names()
df_dictionary = pd.DataFrame(feature_names)
dictionary = df_dictionary.to_dict()[0]
# df_doc_term_matrix = pd.DataFrame(doc_term_matrix.toarray())
term_doc_matrix = doc_term_matrix.transpose()

In [156]:
def create_gensim_lsa_model(number_of_topics,words, dictionary, doc_term_matrix):
    # generate LSA model
    lsamodel = LsiModel(doc_term_matrix, num_topics=number_of_topics, id2word = dictionary)  # train model
    print(lsamodel.print_topics(num_topics=-1, num_words=words))
    return lsamodel

In [ ]:
from sklearn.utils.extmath import randomized_svd

U, Sigma, VT = randomized_svd(doc_term_matrix, 
                              n_components=15,
                              n_iter=5,
                              random_state=42)

In [157]:
model = create_gensim_lsa_model(10, 100, dictionary, term_doc_matrix)

[(0, '0.574*"help" + 0.321*"new" + 0.263*"world" + 0.160*"game" + 0.149*"support" + 0.143*"make" + 0.141*"book" + 0.134*"need" + 0.127*"life" + 0.123*"people" + 0.115*"music" + 0.109*"art" + 0.104*"time" + 0.099*"free" + 0.094*"series" + 0.093*"community" + 0.086*"way" + 0.085*"home" + 0.084*"create" + 0.083*"film" + 0.082*"bring" + 0.082*"app" + 0.081*"power" + 0.080*"camera" + 0.079*"smart" + 0.078*"love" + 0.077*"best" + 0.077*"children" + 0.077*"based" + 0.075*"fund" + 0.073*"album" + 0.071*"project" + 0.070*"high" + 0.070*"women" + 0.069*"funds" + 0.069*"food" + 0.068*"designed" + 0.067*"photography" + 0.067*"design" + 0.066*"play" + 0.066*"raise" + 0.065*"story" + 0.063*"water" + 0.063*"let" + 0.062*"wireless" + 0.062*"money" + 0.060*"build" + 0.060*"needs" + 0.060*"like" + 0.059*"want" + 0.059*"unique" + 0.058*"save" + 0.057*"space" + 0.057*"bike" + 0.056*"quality" + 0.056*"portable" + 0.056*"better" + 0.055*"campaign" + 0.054*"use" + 0.054*"light" + 0.054*"work" + 0.054*"fun" +